# Implementation of QAOA

### Sources:
Quantum Circuit:

https://pennylane.ai/qml/demos/tutorial_QUBO

https://pennylane.ai/qml/demos/tutorial_qaoa_intro

Portoflio optimization:

https://medium.com/mdr-inc/portfolio-optimization-with-minimum-risk-using-qaoa-e29e1d66c194

https://www.nature.com/articles/s41598-023-45392-w

In [1]:
# Import functions
import qaoa_functions, importlib
importlib.reload(qaoa_functions)

from qaoa_functions import (
    qubo_equality_contraint,
    qubo_unbalanced_penalty,
    QUBO_to_Ising,
    qaoa_circuit,
    qaoa_expectation,
    optimize_qaoa_parameters,
    decode_portfolio_allocation,
    calculate_portfolio_risk,
    valid_top_portfolios_infos,
    top_portfolios_by_energy
)

### Importing libraries

In [2]:
import yfinance as yf
import pandas as pd

import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from collections import defaultdict
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

import scipy.stats as st

### Setting global parameters

In [3]:
#tickers = ['AAPL', 'AMZN', 'GOOGL', 'PG', 'JPM', 'UNH']
tickers = ['GLD', '^GSPC']
n_assets = len(tickers)
budget = 1.0
bits_per_asset = 7
p = 2

In [17]:
# Define QAOA parameters - recommended params for two assets
gammas = np.linspace(0.1, 1.0, p)  # Params for cost Hamiltonian
betas = np.linspace(0.1, 0.8, p)[::-1]  # Params for mixer Hamiltonian

In [5]:
# Set up simulator
simulator = AerSimulator()

In [46]:
# Simulated:
tickers = ['Return 50', 'Risk 50']
n_assets = len(tickers)
budget = 1.0
bits_per_asset = 3
p = 2
returns_list = np.array([0.5, 0.0])  # returns for asset 1, 2 and 3
cov_matrix = np.array([[0.0,  0.0 ],
                       [0.0,  0.25]])


### Without optimization

In [47]:
# Penalities and coefficients
return_coefficient = 1
lambd_equality = 20
shots_number = 5000

# Construct the QUBO matrix
Q = qubo_equality_contraint(cov_matrix, n_assets, bits_per_asset, budget, returns_list, return_coeff=return_coefficient, lambd=lambd_equality)
offset = 0
h, J, offset = QUBO_to_Ising(Q, offset)

total_qubits = n_assets * bits_per_asset # all possible allocations for each asset

# Create the QAOA circuit
qc = qaoa_circuit(gammas, betas, h, J, total_qubits)

In [48]:
# return top n 'decoded' portfolios with all relative additional info
result = simulator.run(qc, shots=shots_number).result()
result_counts = result.get_counts()
valid_p = valid_top_portfolios_infos(result_counts, n_assets, bits_per_asset, cov_matrix, tickers, top_n=5, tolerance=1)
energy_portfolios = top_portfolios_by_energy(result_counts, h, J, offset, n_assets, bits_per_asset, cov_matrix, tickers, top_n=5)


In [49]:
energy_portfolios

[{'bitstring': '11000001111111',
  'energy': np.float64(-20.491073608398438),
  'risk': np.float64(0.0001373291015625),
  'volatility': np.float64(0.01171875),
  'allocations': {0: 0.9921875, 1: 0.0234375},
  'total_allocation': 1.015625,
  'count': 1},
 {'bitstring': '00100000111111',
  'energy': np.float64(-20.487060546875),
  'risk': np.float64(0.000244140625),
  'volatility': np.float64(0.015625),
  'allocations': {0: 0.984375, 1: 0.03125},
  'total_allocation': 1.015625,
  'count': 1},
 {'bitstring': '01000001011111',
  'energy': np.float64(-20.48699951171875),
  'risk': np.float64(6.103515625e-05),
  'volatility': np.float64(0.0078125),
  'allocations': {0: 0.9765625, 1: 0.015625},
  'total_allocation': 0.9921875,
  'count': 2},
 {'bitstring': '00100001011111',
  'energy': np.float64(-20.48681640625),
  'risk': np.float64(0.000244140625),
  'volatility': np.float64(0.015625),
  'allocations': {0: 0.9765625, 1: 0.03125},
  'total_allocation': 1.0078125,
  'count': 1},
 {'bitstring

### With optimization

In [58]:
# Penalities and coefficients
return_coefficient = 1
lambd_equality = 20

# Construct the QUBO matrix
Q = qubo_equality_contraint(cov_matrix, n_assets, bits_per_asset, budget, returns_list, return_coeff=return_coefficient, lambd=lambd_equality)
offset = 0
h, J, offset = QUBO_to_Ising(Q, offset)

In [59]:
shots_per_eval = 5000  # Number of shots per circuit evaluation
maxiter = 100  # Maximum number of optimization iterations

# Run the optimization
# optimize_qaoa_parameters(h, J, num_qubits, n_assets, bit_per_asset, cov_matrix, p, shots=5000, maxiter=100)
opt_result = optimize_qaoa_parameters(h, J, total_qubits, p, shots_per_eval, maxiter)
optimized_gammas = opt_result.x[:p]
optimized_betas = opt_result.x[p:]

In [60]:
# Print results
print("\nOptimization Results:")
print(f"Message: {opt_result.message}")
print(f"Final cost function value: {opt_result.fun}")
print(f"Number of function evaluations: {opt_result.nfev}")


Optimization Results:
Message: Optimization terminated successfully.
Final cost function value: -0.7651642303466797
Number of function evaluations: 32


In [61]:
# Compare with initial results

# qaoa_expectation(gammas, betas, h, J, num_qubits, n_assets, bits_per_asset, cov_matrix, shots=5000)
initial_risk = qaoa_expectation(np.concat([gammas, betas]), h, J, total_qubits, shots=5000)
final_risk = qaoa_expectation(np.concat([optimized_gammas, optimized_betas]), h, J, total_qubits, shots=5000)
print(f"\nRisk Comparison:")
print(f"Initial Risk: {initial_risk:.6f}")
print(f"Final Risk: {final_risk:.6f}")
print(f"Improvement: {((initial_risk - final_risk) / initial_risk * 100):.2f}%")


Risk Comparison:
Initial Risk: -0.499612
Final Risk: -0.755053
Improvement: -51.13%


In [62]:
print("\nOptimal parameters:")
print("Gammas:", [f"{gamma:.4f}" for gamma in opt_result.x[:p]])
print("Betas:", [f"{beta:.4f}" for beta in opt_result.x[p:]])


Optimal parameters:
Gammas: ['0.1001', '1.0000']
Betas: ['0.7999', '0.2001']


In [63]:
# Run the circuit with optimized parameters and increased number of shots for final evaluation
final_shots = 5000  # Increased shots for final evaluation
qc_optimized = qaoa_circuit(optimized_gammas, optimized_betas, h, J, total_qubits)
result_optimized = simulator.run(qc_optimized, shots=final_shots).result()
counts_optimized = result_optimized.get_counts()

In [64]:
# Analyze results with optimized parameters
valid_p_optimized = valid_top_portfolios_infos(counts_optimized, n_assets, bits_per_asset, cov_matrix, tickers, top_n=5, tolerance=0.1)

energy_portfolios_optimized = top_portfolios_by_energy(counts_optimized, h, J, offset, n_assets, bits_per_asset, cov_matrix, tickers, top_n=5)

print("\nTop portfolios with optimized parameters:")
print(f"(Evaluated with {final_shots} shots)")
for i, portfolio in enumerate(energy_portfolios_optimized, 1):
    print(f"\nPortfolio {i}")
    print(f"Count: {portfolio['count']}, bitstring: {portfolio['bitstring']}")
    print("Allocations:")
    for j, ticker in enumerate(tickers):
        print(f"{ticker}: {portfolio['allocations'].get(j, 0) * 100:.2f}%")
    print(f"Portfolio Risk: {portfolio['risk']:.6f}")
    print(f"Portfolio Volatility: {portfolio['volatility']:.6f}")


Top portfolios with optimized parameters:
(Evaluated with 5000 shots)

Portfolio 1
Count: 1, bitstring: 10000001111111
Allocations:
Return 50: 99.22%
Risk 50: 0.78%
Portfolio Risk: 0.000015
Portfolio Volatility: 0.003906

Portfolio 2
Count: 2, bitstring: 01000001111111
Allocations:
Return 50: 99.22%
Risk 50: 1.56%
Portfolio Risk: 0.000061
Portfolio Volatility: 0.007812

Portfolio 3
Count: 1, bitstring: 11000000111111
Allocations:
Return 50: 98.44%
Risk 50: 2.34%
Portfolio Risk: 0.000137
Portfolio Volatility: 0.011719

Portfolio 4
Count: 1, bitstring: 11000001011111
Allocations:
Return 50: 97.66%
Risk 50: 2.34%
Portfolio Risk: 0.000137
Portfolio Volatility: 0.011719

Portfolio 5
Count: 1, bitstring: 00000000111111
Allocations:
Return 50: 98.44%
Risk 50: 0.00%
Portfolio Risk: 0.000000
Portfolio Volatility: 0.000000


### Functions

In [43]:
# Penalities and coefficients

max_return = np.sum(np.abs(returns_list))
max_risk = np.trace(cov_matrix) * n_assets
return_coefficient = max_risk / max_return  # 1.08 in this case

lambd_equality = 20

lambd_under = 5  # under-alloc
lambd_over = 20  # over-alloc

In [44]:
# Compare traditional vs unbalanced penalization
# Traditional approach
Q_traditional = qubo_equality_contraint(cov_matrix, n_assets, bits_per_asset, budget, returns_list, return_coeff=return_coefficient, lambd=lambd_equality)
h_trad, J_trad, offset_trad = QUBO_to_Ising(Q_traditional, offset)

# Unbalanced penalization approach
 # Stronger penalty for over-allocation
Q_unbalanced = qubo_unbalanced_penalty(cov_matrix, n_assets, bits_per_asset, budget, returns_list, return_coeff=return_coefficient, 
                                     lambd_under=lambd_under, lambd_over=lambd_over)
h_unbal, J_unbal, offset_unbal = QUBO_to_Ising(Q_unbalanced, offset)

# Run optimization with both approaches
print("Traditional Penalization Results:")
opt_result_trad = optimize_qaoa_parameters(h_trad, J_trad, total_qubits, p, shots=shots_per_eval, maxiter=maxiter)
print(f"Final cost: {opt_result_trad.fun:.6f}")

print("\nUnbalanced Penalization Results:")
opt_result_unbal = optimize_qaoa_parameters(h_unbal, J_unbal, total_qubits, p, shots=shots_per_eval, maxiter=maxiter)
print(f"Final cost: {opt_result_unbal.fun:.6f}")

# Evaluate final portfolios with more shots
final_shots = 5000

# Traditional approach results
qc_trad = qaoa_circuit(opt_result_trad.x[:p], opt_result_trad.x[p:], h_trad, J_trad, total_qubits)
result_trad = simulator.run(qc_trad, shots=final_shots).result()
counts_trad = result_trad.get_counts()
valid_p_trad = valid_top_portfolios_infos(counts_trad, n_assets, bits_per_asset, cov_matrix, tickers, top_n=3, tolerance=1)

energy_portfolios_trad = top_portfolios_by_energy(counts_trad, h_trad, J_trad, offset_trad, n_assets, bits_per_asset, cov_matrix, tickers, top_n=3)

# Unbalanced approach results
qc_unbal = qaoa_circuit(opt_result_unbal.x[:p], opt_result_unbal.x[p:], h_unbal, J_unbal, total_qubits)
result_unbal = simulator.run(qc_unbal, shots=final_shots).result()
counts_unbal = result_unbal.get_counts()
valid_p_unbal = valid_top_portfolios_infos(counts_unbal, n_assets, bits_per_asset, cov_matrix, tickers, top_n=3, tolerance=1)

energy_portfolios_unbal = top_portfolios_by_energy(counts_unbal, h_unbal, J_unbal, offset_unbal, n_assets, bits_per_asset, cov_matrix, tickers, top_n=3)


print("\nTop 3 Portfolios - Traditional Approach:")
for i, portfolio in enumerate(energy_portfolios_trad, 1):
    print(f"\nPortfolio {i}")
    print(f"Count: {portfolio['count']}, bitstring: {portfolio['bitstring']}")
    print("Allocations:")
    for j, ticker in enumerate(tickers):
        print(f"{ticker}: {portfolio['allocations'].get(j, 0) * 100:.2f}%")
    print(f"Portfolio Risk: {portfolio['risk']:.6f}")
    print(f"Energy: {portfolio['energy']:.6f}")
    print(f"Total Allocation: {portfolio['total_allocation']:.4f}")

print("\nTop 3 Portfolios - Unbalanced Approach:")
for i, portfolio in enumerate(energy_portfolios_unbal, 1):
    print(f"\nPortfolio {i}")
    print(f"Count: {portfolio['count']}, bitstring: {portfolio['bitstring']}")
    print("Allocations:")
    for j, ticker in enumerate(tickers):
        print(f"{ticker}: {portfolio['allocations'].get(j, 0) * 100:.2f}%")
    print(f"Energy: {portfolio['energy']:.6f}")
    print(f"Portfolio Risk: {portfolio['risk']:.6f}")
    print(f"Total Allocation: {portfolio['total_allocation']:.4f}")


Traditional Penalization Results:
Final cost: -6.752788

Unbalanced Penalization Results:
Final cost: -5.313669

Top 3 Portfolios - Traditional Approach:

Portfolio 1
Count: 1, bitstring: 110010000010110
Allocations:
Return 50: 40.62%
Risk 50: 0.00%
Return 75 - Risk 45: 59.38%
Portfolio Risk: 0.071389
Energy: -26.543665
Total Allocation: 1.0000

Portfolio 2
Count: 1, bitstring: 000110000000010
Allocations:
Return 50: 25.00%
Risk 50: 0.00%
Return 75 - Risk 45: 75.00%
Portfolio Risk: 0.113906
Energy: -26.543570
Total Allocation: 1.0000

Portfolio 3
Count: 2, bitstring: 010010000001110
Allocations:
Return 50: 43.75%
Risk 50: 0.00%
Return 75 - Risk 45: 56.25%
Portfolio Risk: 0.064072
Energy: -26.542497
Total Allocation: 1.0000

Top 3 Portfolios - Unbalanced Approach:

Portfolio 1
Count: 2, bitstring: 001010000010110
Allocations:
Return 50: 40.62%
Risk 50: 0.00%
Return 75 - Risk 45: 62.50%
Energy: -21.546757
Portfolio Risk: 0.079102
Total Allocation: 1.0312

Portfolio 2
Count: 1, bitstring:

In [50]:
from qaoa_functions import brute_force_ground_state

# build Q, convert to Ising exactly as before
ground = brute_force_ground_state(h_trad, J_trad, offset_trad,
                                  n_assets, bits_per_asset,
                                  cov_matrix)

print("Exact ground state(s) of the Ising Hamiltonian:")
for s in ground:
    print(s)

IndexError: list index out of range

In [21]:
[2**(-i-1) for i in range(bits_per_asset)][::-1]

[0.03125, 0.0625, 0.125, 0.25, 0.5]